In [37]:
import cv2,os
import numpy as np

subjects = ["","s1", "s2", "s3","s4"]

In [38]:
def detect_face(img):

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface.xml')
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=2);
    
    if (len(faces) == 0):
        return None, None
    faceass=[]
    grayass=[]
    for j,(x,y,w,h) in enumerate(faces):
        faceass.append(faces[j])
        grayass.append(gray[y:y+w, x:x+h])

    return grayass, faceass

In [39]:
def prepare_training_data():
    dirs = os.listdir("servmedir")
    
    faces = []
    labels = []
    
    for dir_name in dirs:
        
        if not dir_name.startswith("s"):
            continue;
        
        label = int(dir_name.replace("s",""))
        
        subject_dir_path = "trainidt"+"/"+dir_name
        
        subject_images_names = os.listdir(subject_dir_path)
        
        for image_name in subject_images_names:
            if image_name.startswith("."):
                continue;
            
            image_path = subject_dir_path + "/" + image_name
            
            face = cv2.imread(image_path)
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            if face is not None:
                faces.append(face)
                labels.append(label)
    return faces, labels

faces, labels = prepare_training_data()

face_recognizer = cv2.face.createLBPHFaceRecognizer()

In [40]:
face_recognizer.train(faces, np.array(labels))

In [41]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)


def draw_text(img, text, rect):
    (x,y,w,h)=rect
    cv2.putText(img, text, (x+(w//2)-15, y-5), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)

In [42]:
def predict(test_img):
    img = test_img.copy()
    
    faces, rects = detect_face(img)
    
    if faces is None:
        return None
    
    for i,face in enumerate(faces):
        rect = rects[i]
        label,predict= face_recognizer.predict(face)
        
        label_text = subjects[label]
        
        draw_rectangle(img, rect)
        
        draw_text(img, label_text, rect)
    
    return img

In [ ]:
# test_img1 = cv2.imread("t1.jpg")
 
# img1 = predict(test_img1)
# img2 = predict(test_img2)
# print("Prediction complete")

# scale_percent = 35 # percent of original size
# width = int(img1.shape[1] * scale_percent / 100)
# height = int(img1.shape[0] * scale_percent / 100)
# dim = (width, height)
 
# # resize image
# img4 = cv2.resize(img1, dim, interpolation = cv2.INTER_AREA)
# img5 = cv2.resize(img2, dim, interpolation = cv2.INTER_AREA)

# cv2.imshow(subjects[1], img4)
# cv2.imshow(subjects[2], img5)
# cv2.waitKey(0)

In [44]:
# cv2.destroyAllWindows()

cap = cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    nface = predict(frame)
    if nface is None:
        cv2.imshow('live',frame)
    else:
        cv2.imshow('live',nface)
    if cv2.waitKey(1)== 10:#Enter key to check keypress code use print(cv2.waitkey())and press key after imshow
        break
cap.release()
cv2.destroyAllWindows()